In [1]:
1

1

fixation using deepcopy

In [2]:
deepcopy(10)

10

In [3]:
# thrdnum = 8
thrdnum = 8
thrdnum

8

In [4]:
using Distributed
addprocs(thrdnum)

8-element Vector{Int64}:
 2
 3
 4
 5
 6
 7
 8
 9

In [5]:
@everywhere using Mongoc
@everywhere using Dates

In [6]:
@everywhere using JSON
@everywhere using JLD2
@everywhere using HDF5
@everywhere using Profile

In [7]:
@everywhere using Dates
@everywhere using Tables
@everywhere using DataFrames
@everywhere using CSV
@everywhere using XLSX
@everywhere using BSON
@everywhere using Distributed
@everywhere using FileIO
# using Printf
# using Query

# addprocs(8)

In [8]:
@everywhere using DataFramesMeta

In [9]:

# Set([1,2])

In [10]:
@everywhere f_dis = "../dicts/code2location.csv"
@everywhere df_dis = @chain CSV.read(f_dis, header=1, DataFrame) begin
end
first(df_dis, 3)
# @everywhere const c2_limit = Set(df_iy[!, "c2"])
# length(c2_limit)

Row,Column1,org_code,icd_10_1,icd_10_3dig,location,dis_name
,Int64,Int64,String7,String3,String,String
1,1,8847269,D460,D46,Myelodysplastic syndromes,１系統に異形成を伴う不応性血球減少症
2,2,8847270,D466,D46,Myelodysplastic syndromes,５ｑ－症候群
3,3,8847271,C847,C84,Non-Hodgkin lymphoma,ＡＬＫ陰性未分化大細胞リンパ腫


In [11]:
@everywhere diss_limit = collect(Set(df_dis[!,"location"]))
@everywhere dis_limit = push!(diss_limit, "multiple", "NoCancers")

dis_limit

63-element Vector{String}:
 "Brain, nervous system"
 "Kaposi sarcoma"
 "Ovary"
 "Renal pelvis"
 "Uterus unspecified"
 "Lung (incl. trachea and bronchus)"
 "Lip"
 "Tonsil"
 "Mesothelioma"
 "Testis"
 "Other thoracic organs"
 "Pancreas"
 "Lymphoid leukaemia"
 ⋮
 "Bone"
 "Other and unspecified"
 "Thyroid"
 "Penis"
 "Connective and soft tissue"
 "Multiple myeloma"
 "Mouth"
 "Other skin"
 "Small intestine"
 "Kidney"
 "multiple"
 "NoCancers"

In [12]:
@everywhere f_iy = "../dicts/Jm01_dis_parse/dfmeta_iy_20231122_150053_929635.csv"
@everywhere df_iy = @chain CSV.read(f_iy, header=1,types=Dict("c2"=>String), DataFrame) begin
    @transform(:yj2 = first.(:c31, 2))
    @subset(in.(:yj2, Ref(["42", "43"])))
    # @subset(in.(:c2, Ref(["662450004"])))
    # @subset(:c2 == "662450004")
end

@everywhere const c2_limit = Set(df_iy[!, "c2"])
length(c2_limit)

1006

In [13]:
first(df_iy)

Row,Column1,c0,c2,c4,c9,c10,c13,c14,c15,c16,c31,ymd,next_month_day1,next_month_day2,filename,dt_rank_neg,yj2
,Int64,Int64,String,String,String15,Float64?,Float64,Float64,Float64,Float64,String15,Date,Date,Date,String31,Float64,String15
1,295826,5,610407022,フェアストン錠４０ ４０ｍｇ,錠,1.0,0.0,0.0,0.0,0.0,4291007F1021,2023-04-01,2023-05-01,2023-05-02,y_ALL20230303.csv,1.0,42


In [14]:
# @everywhere df_iy
# @everywhere x = df_iy[!, "c2"]
# @everywhere const x2 = [i for i in x]

In [15]:
# @everywhere const c2_limit = c2_limit_org

In [16]:
@everywhere function gen_dict()
    f_mst = "../dicts/Jm01_dis_parse/byomei_cancer_20210701.csv"
    df_mst = @chain CSV.read(f_mst, header=0,types=Dict(4=>String), DataFrame) begin
            @transform(:tag = first.(:Column17, 3))
        end
    d_dis2code2 = Dict{String, String}()
    for r in eachrow(df_mst)
        d_dis2code2[r["Column4"]] = r["tag"]
        # print(r["Column4"])
        # break
    end


        
    # d_cs = Dict{Vector{String}, Dict{String, Dict}}()
    d_cs = Dict{String, Dict{String, Dict}}()
    d_ids = Dict{String, Dict{String, Any}}()
    
    d_ids_each = Dict{String, Any}()
    d_ids_each["first_ym"] = 0
    d_ids_each["gender"] = 0
    d_ids_each["first_cancers"] = []
    d_ids_each["ym_count"] = 0
    d_ids_each["last_ym"] = 0

    d_ym = Dict{Int32, Float32}()
    d_ymd = Dict{Int32, Float32}()
    for y in 2011:2022
        # println(string(y))
        for m in 1:12
            ym = string(y) * lpad(m, 2, "0")
            d_ym[parse(Int32, ym)] = 0.0
            # println(lpad(m, 2, "0"))
            for d in 1:31
                ymd = ym * lpad(d, 2, "0")
                d_ymd[parse(Int32, ymd)] = 0.0
                # println(lpad(d, 2, "0"))
            end
        end
    end

    f_iy = "../dicts/Jm01_dis_parse/dfmeta_iy_20231122_150053_929635.csv"
    df_iy = @chain CSV.read(f_iy, header=1,types=Dict("c2"=>String), DataFrame) begin
                @transform(:yj2 = first.(:c31, 2))
                @subset(in.(:yj2, Ref(["42", "43"])))
            end
    d_iy = Dict{String, Dict{Int32, Float32}}()
    for r in eachrow(df_iy)
        d_iy[r["c2"]] = deepcopy(d_ymd)
    end

    for c in dis_limit
        d_cs[c] = deepcopy(d_iy)
    end
    println("base dicts generated")
    return d_cs, d_ids, d_ids_each, d_ym, d_ymd, d_dis2code2, d_iy
end

In [17]:
# may cause memory flux
@everywhere d_cs, d_ids, d_ids_each, d_ym, d_ymd, d_dis2code2, d_iy = gen_dict()

base dicts generated
      From worker 9:	base dicts generated
      From worker 5:	base dicts generated
      From worker 7:	base dicts generated
      From worker 6:	base dicts generated
      From worker 2:	base dicts generated
      From worker 3:	base dicts generated
      From worker 8:	base dicts generated
      From worker 4:	base dicts generated


In [18]:
# d_iy

In [19]:
# const c_ids_each = d_ids_each
# const c_dis2code2 = d_dis2code2 
# const c_ym = d_ym
# const c_ymd = d_ymd
# const c_iy = d_iy

In [20]:
@everywhere const dbs = ["s_" * lpad(i, 3, '0') for i in 0:255]

In [21]:
min(1,3)

1

In [22]:
max(1,2,4)

4

here must be fixed to each code only...

Done!


In [23]:
# d_cs

In [24]:
occursin("_", "aiu_")

true

In [25]:
# d_iy

In [26]:
# @everywhere 
@everywhere function e2d_add_use(d_cs, d_ids, d_ids_each, d_ym, d_ymd, d_dis2code2, d_iy, e, d1)
    #get cancer
    # cancers = e["cancers"]
    id1n = e["id1n"]
    cancers = d1[id1n]
    
    if occursin("_", cancers)
        cancers = "multiple"
    elseif cancers == ""
        cancers = "NoCancers"
    end

    
    # if length(cancers) == 0
    #     cancers = ["NoCancers"]
    # end 
    
    ym = e["ym"]
    ym_s =string(ym)

    # if !haskey(d_cs, cancers)
    #     d_cs[cancers] = deepcopy(d_iy)
    # end
    
    for (dc,dv) in e["usage"]["iy"]
        # dc: drug code, dv: dict k=date(01-31), v:value
        # if !haskey(d_cs[cancers], dc)
        #     d_cs[cancers][dc] = copy(d_ymd)
        # end
        
        for (k, v) in dv
            # ym_s:"202010", k :"03" etc
            if dc in c2_limit
                # if ! haskey(d_cs[cancers], dc)
                #     d_cs[cancers][dc] = deepcopy(d_ymd)
                # end
                # if parse(Int32, ym_s * k) == 20130231
                #     println(id1n)
                #     println(ym_s)
                #     println(parse(Int32, ym_s * k))
                #     println(dc)
                #     println(v)
                # end
                d_cs[cancers][dc][parse(Int32, ym_s * k)] += v
            end
        end
    end
    
end

In [27]:
# f = "s_001"
# f_in1 = "../../../storage/d_id1c/id1ns_d_"*f*".jld2"
# f_in2 = "../../../storage/smr_tabl/smr_tbl_id1ns_"*f*".jld2"
# d1 = load(f_in1)["data"]
# d2 = load(f_in2)["data"]

In [28]:
# d1

In [29]:
# reshape(dbs[1:4], thrdnum, :)

In [30]:
# @everywhere 
const dbs2d = reshape(dbs, thrdnum, :)
# const dbs2d = reshape(dbs[1:8], thrdnum, :)

8×32 Matrix{String}:
 "s_000"  "s_008"  "s_016"  "s_024"  …  "s_224"  "s_232"  "s_240"  "s_248"
 "s_001"  "s_009"  "s_017"  "s_025"     "s_225"  "s_233"  "s_241"  "s_249"
 "s_002"  "s_010"  "s_018"  "s_026"     "s_226"  "s_234"  "s_242"  "s_250"
 "s_003"  "s_011"  "s_019"  "s_027"     "s_227"  "s_235"  "s_243"  "s_251"
 "s_004"  "s_012"  "s_020"  "s_028"     "s_228"  "s_236"  "s_244"  "s_252"
 "s_005"  "s_013"  "s_021"  "s_029"  …  "s_229"  "s_237"  "s_245"  "s_253"
 "s_006"  "s_014"  "s_022"  "s_030"     "s_230"  "s_238"  "s_246"  "s_254"
 "s_007"  "s_015"  "s_023"  "s_031"     "s_231"  "s_239"  "s_247"  "s_255"

In [31]:
string(length(dbs2d[1,:]))

"32"

In [32]:
@everywhere function create_dicts_iy(ss)

    a = now()
    println(a)
    client = Mongoc.Client("192.168.0.25", 27017)
    
    db = client["smr"]
    # may cause memory flux
    # d_cs, d_ids, d_ids_each, d_ym, d_ymd, d_dis2code2, d_iy = gen_dict()

    length_ss_str = string(length(ss))
    ss_start = ss[1]
    file_tag = ss_start * "_" * length_ss_str
    f_out = "../../../storage/smr_iy_tabl_fix/smr_tbl_cancer_iy_dict_"*file_tag*"_dpfix.jld2"
    println(f_out)
    
    for s in ss
        println(s)

        #the dict id1n -> cancer categ
        f_in1 = "../../../storage/d_id1c/id1ns_d_"*s*".jld2"
        
        d1 = load(f_in1)["data"]
        # d2 = load(f_in2)["data"]
        collection = db[s]

        a2 = now()
        for (i,e) in enumerate(Mongoc.find(collection))
            e2d_add_use(d_cs, d_ids, d_ids_each, d_ym, d_ymd, d_dis2code2, d_iy,e, d1)
            if i % 1e+6 == 0
               # break
                print(s)
                print(" checkpoint:")
                print(i)
                b = now()
            
                print(" time:")
                print(canonicalize(b -a))
                print(" chunktime:")
                print(canonicalize(b -a2))
                # print(" dictsize:")
                # print(length(d_id1n))
                print("\n")
                a2 = now()
            end
        
        end
        print(s)
        print(", endpoint:")
        
        b = now()
        print(" time:")
        print(canonicalize(b -a))
        print(" chunktime:")
        print(canonicalize(b -a2))
        print("\n")
        a2 = now()
        # filepath  =get_path_id1s(s)
        # save(get_path_id1s(s),"data", d_ids)
    end

    
    # f = get_path_iy("whole")
    a2 = now()
    print(":loop finished:")
    println(a2)
    save(f_out,"data", d_cs)
    # save(f_out,"data", d_iy)
    # save(get_path_id1s("whole"),"data", d_ids)
    return 1
end

In [33]:
# get_path(x1) = "../../../storage/smr_tabl/smr_tbl_$x1.jld2"
# @everywhere  
# get_path_cs(x1) = "../../../storage/smr_tabl/smr_tbl_cs_$x1.jld2"
# @everywhere  get_path_id1s(x1) = "../../../storage/smr_tabl/smr_tbl_id1ns_$x1.jld2"
# get_path_iy(x1) = "../../../storage/smr_tabl/smr_tbl_iy_$x1.jld2"
# get_path_cs("aaa")
# get_path_id1s("aaa")

In [34]:
# create_dicts_iy(dbs)

In [35]:
# results = @distributed (+) for dbi in dbs[1:4]
#     # [250:256]
#     # get_d(dbxs)
#     create_dicts_iy([dbi])
    
#     # println(dbxs)
# end

In [36]:
results = @distributed (+) for i=1:size(dbs2d, 1)
    ss = dbs2d[i,:]
    create_dicts_iy(ss)
    
    # println(dbxs)
end

      From worker 7:	2024-11-14T14:40:41.449
      From worker 7:	../../../storage/smr_iy_tabl_fix/smr_tbl_cancer_iy_dict_s_005_32_dpfix.jld2
      From worker 7:	s_005
      From worker 5:	2024-11-14T14:40:41.450
      From worker 5:	../../../storage/smr_iy_tabl_fix/smr_tbl_cancer_iy_dict_s_003_32_dpfix.jld2
      From worker 5:	s_003
      From worker 3:	2024-11-14T14:40:41.458
      From worker 3:	../../../storage/smr_iy_tabl_fix/smr_tbl_cancer_iy_dict_s_001_32_dpfix.jld2
      From worker 3:	s_001
      From worker 6:	2024-11-14T14:40:41.520
      From worker 6:	../../../storage/smr_iy_tabl_fix/smr_tbl_cancer_iy_dict_s_004_32_dpfix.jld2
      From worker 6:	s_004
      From worker 2:	2024-11-14T14:40:41.519
      From worker 2:	../../../storage/smr_iy_tabl_fix/smr_tbl_cancer_iy_dict_s_000_32_dpfix.jld2
      From worker 2:	s_000
      From worker 9:	2024-11-14T14:40:41.730
      From worker 9:	../../../storage/smr_iy_tabl_fix/smr_tbl_cancer_iy_dict_s_007_32_dpfix.jld2
      From wo

8

In [40]:
results

8

In [37]:
kokomade

LoadError: UndefVarError: kokomade not defined

In [38]:
# KeyError: key "662450004" not found

In [39]:
 1

1